<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

-   Understand the dataset for Chicago Public School level performance
-   Store the dataset in an Db2 database on IBM Cloud instance
-   Retrieve metadata about tables and columns and query data from mixed case columns
-   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01&download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv">link</a>.

**NOTE**:

For the learners who are encountering issues with loading from .csv in DB2 on Firefox, you can download the .txt files and load the data with those: <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.txt">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01"><img src="https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database

The following modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you may need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [1]:
# These libraries are pre-installed in SN Labs. If running in another environment please uncomment lines below to install them:
# !pip install --force-reinstall ibm_db==3.1.0 ibm_db_sa==0.3.3
# Ensure we don't load_ext with sqlalchemy>=1.4 (incompadible)
# !pip uninstall sqlalchemy==1.4 -y && pip install sqlalchemy==1.3.24
# !pip install ipython-sql

In [2]:
%load_ext sql

In [3]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
%sql ibm_db_sa://qvf69173:ayVZldqZuWHGbNI7@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb?security=SSL

'Connected: qvf69173@bludb'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [4]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql SELECT * FROM syscat.tables WHERE TABSCHEMA='QVF69173'

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
QVF69173,PETSALE,QVF69173,U,T,N,None,None,None,None,2023-02-20 00:21:42.359240,2023-02-20 04:58:07.868533,2023-02-20 04:58:07.868538,2023-02-20 04:57:22.787684,5,4,4771,10,1,0,1,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,54,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2023-02-20 00:21:42.359240,0,,None,None,N,QVF69173,,,,2023-02-20,,N,R,N,-1.0,None
QVF69173,EMPLOYEES,QVF69173,U,T,N,None,None,None,None,2023-02-20 05:17:42.434140,2023-02-20 05:17:42.537624,2023-02-20 05:17:42.537629,2023-02-20 05:37:17.849154,11,5,4771,10,1,0,2,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,118,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2023-02-20 05:17:42.434140,0,,None,None,N,QVF69173,,,,2023-02-23,,N,R,N,-1.0,None
QVF69173,JOB_HISTORY,QVF69173,U,T,N,None,None,None,None,2023-02-20 05:17:42.548288,2023-02-20 05:17:42.668756,2023-02-20 05:17:42.668761,2023-02-20 05:47:23.067063,4,6,4771,10,1,0,2,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,2,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,43,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2023-02-20 05:17:42.548288,0,,None,None,N,QVF69173,,,,2023-02-20,,N,R,N,-1.0,None
QVF69173,JOBS,QVF69173,U,T,N,None,None,None,None,2023-02-20 05:17:42.679417,2023-02-20 05:17:42.797751,2023-02-20 05:17:42.797755,2023-02-20 07:32:30.212466,4,7,4771,10,1,0,2,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,53,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2023-02-20 05:17:42.679417,0,,None,None,N,QVF69173,,,,2023-02-22,,N,R,N,-1.0,None
QVF69173,DEPARTMENTS,QVF69173,U,T,N,None,None,None,None,2023-02-20 05:17:42.807992,2023-02-20 05:17:42.927122,2023-02-20 05:17:42.927127,2023-02-20 05:32:59.283717,4,8,4771,3,1,0,2,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,57,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2023-02-20 05:17:42.807992,0,,None,None,N,QVF69173,,,,2023-02-22,,N,R,N,-1.0,None
QVF69173,LOCATIONS,QVF69173,U,T,N,None,None,None,None,2023-02-20 05:17:42.935899,2023-02-20 05:17:43.055110,2023-02-20 05:17:43.055116,2023-02-20 05:47:23.108325,2,9,4771,3,1,0,2,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,2,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,28,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2023-02-20 05:17:42.935899,0,,None,None,N,QVF69173,,,,2023-02-20,,N,R,N,-1.0,None
QVF69173,PETRESCUE,QVF69173,U,T,N,None,None,None,None,2023-02-21 12:55:31.300718,2023-02-21 12:55:31.413325,2023-02-21 12:55:31.413329,2023-02-21 12:56:33.032669,5,10,4771,9,1,0,1,-1,-1,-1,0,QVF69173SPACE1,None,None,0,0,0,1,1,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,No

Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [5]:
%%sql 

SELECT * 
    FROM syscat.columns
    LIMIT 5
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


tabschema,tabname,colname,colno,typeschema,typename,length,scale,typestringunits,stringunitslength,DEFAULT,NULLS,codepage,collationschema,collationname,logged,compact,colcard,high2key,low2key,avgcollen,keyseq,partkeyseq,nquantiles,nmostfreq,numnulls,target_typeschema,target_typename,scope_tabschema,scope_tabname,source_tabschema,source_tabname,dl_features,special_props,hidden,inline_length,pctinlined,IDENTITY,rowchangetimestamp,GENERATED,text,compress,avgdistinctperpage,pagevarianceratio,sub_count,sub_delim_length,avgcollenchar,implicitvalue,seclabelname,rowbegin,rowend,transactionstartid,pctencoded,avgencodedcollen,qualifier,func_path,randdistkey,remarks
SYSIBM,SYSTABLES,NAME,0,SYSIBM,VARCHAR,128,0,OCTETS,128,None,N,1208,SYSIBM,IDENTITY,,,1776,'wlmWorkloadStats','101 DBeaver',16,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,12,None,None,N,N,N,-1,-1.0,None,None,N,None
SYSIBM,SYSTABLES,CREATOR,1,SYSIBM,VARCHAR,128,0,OCTETS,128,None,N,1208,SYSIBM,IDENTITY,,,668,'ZYC22630','BBH47419',12,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,8,None,None,N,N,N,-1,-1.0,None,None,N,None
SYSIBM,SYSTABLES,TYPE,2,SYSIBM,CHARACTER,1,0,OCTETS,1,None,N,1208,SYSIBM,IDENTITY,,,4,'T','S',1,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
SYSIBM,SYSTABLES,CTIME,3,SYSIBM,TIMESTAMP,10,6,None,None,None,N,0,None,None,,,5427,'2023-03-04-06.50.02.947977','2018-03-13-03.17.29.779351',10,None,0,20,10,0,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,-1,None,None,N,N,N,-1,-1.0,None,None,N,None
SYSIBM,SYSTABLES,REMARKS,4,SYSIBM,VARCHAR,254,0,OCTETS,254,None,Y,1208,SYSIBM,IDENTITY,,,4,'version=1.0','Pure Scale Nodes',5,None,0,20,10,5421,None,None,None,None,None,None,None,None,,0,-1,N,N,,None,O,None,-1.0,-1,-1,0,None,None,N,N,N,-1,-1.0,None,None,N,None


In [6]:
%%sql 

SELECT COUNT(colname) 
    FROM syscat.columns
    WHERE tabname = 'CHICAGO_PUBLIC_SCHOOLS'
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


1
78


Double-click **here** for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [7]:
%%sql 

SELECT colname, typename, length
    FROM syscat.columns
    WHERE tabname = 'CHICAGO_PUBLIC_SCHOOLS'
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


colname,typename,length
SCHOOL_ID,INTEGER,4
NAME_OF_SCHOOL,VARCHAR,64
ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL,VARCHAR,2
STREET_ADDRESS,VARCHAR,29
CITY,VARCHAR,7
STATE,VARCHAR,2
ZIP_CODE,INTEGER,4
PHONE_NUMBER,VARCHAR,14
LINK,VARCHAR,78
NETWORK_MANAGER,VARCHAR,40


Double-click **here** for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [8]:
%%sql 

SELECT *
    FROM CHICAGO_PUBLIC_SCHOOLS
    LIMIT 5
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


school_id,name_of_school,elementary__middle__or_high_school,street_address,city,state,zip_code,phone_number,link,network_manager,collaborative_name,adequate_yearly_progress_made_,track_schedule,cps_performance_policy_status,cps_performance_policy_level,healthy_school_certified,safety_icon,safety_score,family_involvement_icon,family_involvement_score,environment_icon,environment_score,instruction_icon,instruction_score,leaders_icon,leaders_score,teachers_icon,teachers_score,parent_engagement_icon,parent_engagement_score,parent_environment_icon,parent_environment_score,average_student_attendance,rate_of_misconducts__per_100_students_,average_teacher_attendance,individualized_education_program_compliance_rate,pk_2_literacy__,pk_2_math__,gr3_5_grade_level_math__,gr3_5_grade_level_read__,gr3_5_keep_pace_read__,gr3_5_keep_pace_math__,gr6_8_grade_level_math__,gr6_8_grade_level_read__,gr6_8_keep_pace_math_,gr6_8_keep_pace_read__,gr_8_explore_math__,gr_8_explore_read__,isat_exceeding_math__,isat_exceeding_reading__,isat_value_add_math,isat_value_add_read,isat_value_add_color_math,isat_value_add_color_read,students_taking__algebra__,students_passing__algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),net_change_explore_and_plan,11th Grade Average ACT (2011),net_change_plan_and_act,college_eligibility__,graduation_rate__,college_enrollment_rate__,college_enrollment,general_services_route,freshman_on_track_rate__,x_coordinate,y_coordinate,latitude,longitude,community_area_number,community_area_name,ward,police_district,location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Weak,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
610281,Adam Clayton Powell Paideia Community Academy Elementary School,ES,7511 S South Shore Dr,Chicago,IL,60649,(773) 535-6650,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610281.pdf,Skyway Elementary Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,54,Strong,66,Strong,74,Very Strong,84,Weak,63,Strong,76,Weak,46,Average,50,95.60%,15.7,95.30%,100.00%,62.4,51.7,21.9,15.1,29,42.8,38.5,27.4,44.8,42.7,14.1,34.4,16.8,16.5,0.7,1.4,Green,Green,17.2,27.3,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,521,46,NDA,1196129.985,1856209.466,41.76032435,-87.55673627,43,SOUTH SHORE,7,4,"(41.76032435, -87.55673627)"
610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago,IL,60652,(773) 535-2280,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610185.pdf,Midway Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 2,No,Strong,61,NDA,NDA,Average,50,Weak,36,Weak,NDA,NDA,NDA,Average,47,Weak,41,95.70%,2.3,94.70%,98.30%,53.7,26.6,38.3,34.7,43.7,57.3,48.8,39.2,46.8,44,7.5,21.9,18.3,15.5,-0.9,-1.0,Red,Red,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,1324,44,NDA,1148427.165,1851012.215,41.74711093,-87.73170248,70,ASHBURN,13,8,"(41.74711093, -87.73170248)"
609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago,IL,60609,(773) 535-4389,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609993.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Track_E,Not on Probation,Level 1,No,Average,56,Average,44,Average,45,Weak,37,Weak,65,Average,48,Average,53,Strong,58,95.50%,10.4,95.80%,100.00%,76.9,NDA,26,24.7,61.8,49.7,39.2,27.2,69.7,60.6,9.1,18.2,11.1,9.6,0.9,2.4,Green,Green,42.9,25,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA

In [18]:
%%sql

SELECT COUNT(school_id)
    FROM CHICAGO_PUBLIC_SCHOOLS
    WHERE ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL = 'ES'
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


1
462


Double-click **here** for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'? ("Elementary School, Middle School, High School")
-->


Double-click **here** for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->


### Problem 2

##### What is the highest Safety Score?


In [20]:
%%sql

SELECT MAX(SAFETY_SCORE)
    FROM CHICAGO_PUBLIC_SCHOOLS
    WHERE ELEMENTARY__MIDDLE__OR_HIGH_SCHOOL = 'ES'
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


1
99


Double-click **here** for a hint

<!--
Use the MAX() function
-->


Double-click **here** for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?


In [22]:
%%sql

SELECT name_of_school, SAFETY_SCORE
    FROM CHICAGO_PUBLIC_SCHOOLS
    WHERE SAFETY_SCORE IN (SELECT MAX(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS)
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,safety_score
Abraham Lincoln Elementary School,99
Alexander Graham Bell Elementary School,99
Annie Keller Elementary Gifted Magnet School,99
Augustus H Burley Elementary School,99
Edgar Allan Poe Elementary Classical School,99
Edgebrook Elementary School,99
Ellen Mitchell Elementary School,99
James E McDade Elementary Classical School,99
James G Blaine Elementary School,99
LaSalle Elementary Language Academy,99


Double-click **here** for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [24]:
%%sql

SELECT name_of_school, AVERAGE_STUDENT_ATTENDANCE
    FROM CHICAGO_PUBLIC_SCHOOLS
    ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC
    LIMIT 10
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,average_student_attendance
Velma F Thomas Early Childhood Center,None
John Charles Haines Elementary School,98.40%
James Ward Elementary School,97.80%
Edgar Allan Poe Elementary Classical School,97.60%
Rachel Carson Elementary School,97.60%
Orozco Fine Arts & Sciences Elementary School,97.60%
Annie Keller Elementary Gifted Magnet School,97.50%
Andrew Jackson Elementary Language Academy,97.40%
Lenart Elementary Regional Gifted Center,97.40%
Disney II Magnet School,97.30%


Double-click **here** for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [25]:
%%sql

SELECT name_of_school, AVERAGE_STUDENT_ATTENDANCE
    FROM CHICAGO_PUBLIC_SCHOOLS
    ORDER BY AVERAGE_STUDENT_ATTENDANCE
    LIMIT 5
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [32]:
%%sql

SELECT name_of_school, REPLACE(AVERAGE_STUDENT_ATTENDANCE, '%', '') AS AVERAGE_STUDENT_ATTENDANCE
    FROM CHICAGO_PUBLIC_SCHOOLS
    ORDER BY AVERAGE_STUDENT_ATTENDANCE
    LIMIT 5
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90
Barbara Vick Early Childhood & Family Center,60.90
Dyett High School,62.50
Wendell Phillips Academy High School,63.00
Orr Academy High School,66.30


Double-click **here** for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->


Double-click **here** for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [49]:
%%sql

SELECT name_of_school, REPLACE(AVERAGE_STUDENT_ATTENDANCE, '%', '') AS AVERAGE_STUDENT_ATTENDANCE
    FROM CHICAGO_PUBLIC_SCHOOLS
    WHERE CAST((REPLACE(AVERAGE_STUDENT_ATTENDANCE, '%', '')) AS DOUBLE) < 70
    ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,average_student_attendance
Roberto Clemente Community Academy High School,69.60
Chicago Vocational Career Academy High School,68.80
Manley Career Academy High School,66.80
Orr Academy High School,66.30
Wendell Phillips Academy High School,63.00
Dyett High School,62.50
Barbara Vick Early Childhood & Family Center,60.90
Richard T Crane Technical Preparatory High School,57.90


Double-click **here** for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->


Double-click **here** for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area


In [80]:
%%sql

SELECT name_of_school, COMMUNITY_AREA_NAME, COLLEGE_ENROLLMENT
    FROM CHICAGO_PUBLIC_SCHOOLS
    WHERE COMMUNITY_AREA_NAME = 'NORTH CENTER'
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,community_area_name,college_enrollment
Albert G Lane Technical High School,NORTH CENTER,4368
Alcott High School for the Humanities,NORTH CENTER,232
Alexander Graham Bell Elementary School,NORTH CENTER,998
DeVry University Advantage Academy High School,NORTH CENTER,200
Friedrich Ludwig Jahn Elementary School,NORTH CENTER,495
John C Coonley Elementary School,NORTH CENTER,668
John J Audubon Elementary School,NORTH CENTER,580


In [82]:
%%sql

SELECT COMMUNITY_AREA_NAME, SUM(COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT, COUNT(name_of_school) AS COUNT_OF_SCHOOLS
    FROM CHICAGO_PUBLIC_SCHOOLS
    GROUP BY COMMUNITY_AREA_NAME
    ORDER BY COLLEGE_ENROLLMENT DESC
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,college_enrollment,count_of_schools
SOUTH LAWNDALE,14793,22
BELMONT CRAGIN,14386,12
AUSTIN,10933,23
GAGE PARK,9915,10
BRIGHTON PARK,9647,8
WEST TOWN,9429,20
HUMBOLDT PARK,8620,13
WEST RIDGE,8197,9
NEAR WEST SIDE,7975,16
NEW CITY,7922,13


Double-click **here** for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->


Double-click **here** for another hint

<!--
Don't forget to group by the Community Area
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [55]:
%%sql

SELECT COMMUNITY_AREA_NAME, SUM(COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT
    FROM CHICAGO_PUBLIC_SCHOOLS
    GROUP BY COMMUNITY_AREA_NAME
    ORDER BY COLLEGE_ENROLLMENT
    LIMIT 5
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,college_enrollment
OAKLAND,140
FULLER PARK,531
BURNSIDE,549
OHARE,786
LOOP,871


Double-click **here** for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->


Double-click **here** for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->


### Problem 10

##### List 5 schools with lowest safety score.


In [56]:
%%sql

SELECT name_of_school, SAFETY_SCORE
    FROM CHICAGO_PUBLIC_SCHOOLS
    ORDER BY SAFETY_SCORE
    LIMIT 5
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


name_of_school,safety_score
Edmond Burke Elementary School,1
Luke O'Toole Elementary School,5
George W Tilton Elementary School,6
Foster Park Elementary School,11
Emil G Hirsch Metropolitan High School,13


Double-click **here** for the solution.

<!-- Solution:

%sql SELECT name_of_school, safety_score \
FROM schools \
ORDER BY safety_score \
LIMIT 5
-->


### Problem 11

##### Get the hardship index for the community area which has College Enrollment of 4368


In [57]:
%sql SELECT * FROM CHICAGO_SOCIOECONOMIC_DATA LIMIT 1

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


index,ca,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,percent_aged_16_unemployed,percent_aged_25_without_high_school_diploma,percent_aged_under_18_or_over_64,per_capita_income_,hardship_index
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0


In [87]:
%%sql

SELECT CPS.COMMUNITY_AREA_NAME, SUM(CPS.COLLEGE_ENROLLMENT) AS TOTAL_ENROLLMENT, MAX(CSD.HARDSHIP_INDEX) AS HARDSHIP_INDEX
    FROM CHICAGO_PUBLIC_SCHOOLS CPS, CHICAGO_SOCIOECONOMIC_DATA CSD
    WHERE UPPER(CPS.COMMUNITY_AREA_NAME) = UPPER(CSD.COMMUNITY_AREA_NAME) AND (SELECT SUM(COLLEGE_ENROLLMENT) FROM CHICAGO_PUBLIC_SCHOOLS) = 4368
    GROUP BY CPS.COMMUNITY_AREA_NAME
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,total_enrollment,hardship_index


In [88]:
%%sql

SELECT CPS.COMMUNITY_AREA_NAME, SUM(CPS.COLLEGE_ENROLLMENT) AS TOTAL_ENROLLMENT, MAX(CSD.HARDSHIP_INDEX) AS HARDSHIP_INDEX
    FROM CHICAGO_PUBLIC_SCHOOLS CPS, CHICAGO_SOCIOECONOMIC_DATA CSD
    WHERE UPPER(CPS.COMMUNITY_AREA_NAME) = UPPER(CSD.COMMUNITY_AREA_NAME) AND COLLEGE_ENROLLMENT = 4368
    GROUP BY CPS.COMMUNITY_AREA_NAME
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,total_enrollment,hardship_index
NORTH CENTER,4368,6.0


In [89]:
%%sql 
select CPS.COMMUNITY_AREA_NAME, cps.college_enrollmenT, hardship_index 
   from chicago_socioeconomic_data CD, CHICAGO_PUBLIC_SCHOOLS CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,college_enrollment,hardship_index
NORTH CENTER,4368,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->


### Problem 12

##### Get the hardship index for the community area which has the school with the  highest enrollment.


In [91]:
%%sql

SELECT CPS.COMMUNITY_AREA_NAME, CPS.NAME_OF_SCHOOL, CPS.COLLEGE_ENROLLMENT, CSD.HARDSHIP_INDEX
    FROM CHICAGO_PUBLIC_SCHOOLS CPS, CHICAGO_SOCIOECONOMIC_DATA CSD
    WHERE UPPER(CPS.COMMUNITY_AREA_NAME) = UPPER(CSD.COMMUNITY_AREA_NAME) AND COLLEGE_ENROLLMENT = (SELECT MAX(COLLEGE_ENROLLMENT) FROM CHICAGO_PUBLIC_SCHOOLS)
;

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


community_area_name,name_of_school,college_enrollment,hardship_index
NORTH CENTER,Albert G Lane Technical High School,4368,6.0


In [92]:
%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from CHICAGO_PUBLIC_SCHOOLS order by college_enrollment desc limit 1 )

 * ibm_db_sa://qvf69173:***@b1bc1829-6f45-4cd4-bef4-10cf081900bf.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32304/bludb
Done.


ca,community_area_name,hardship_index
5.0,North Center,6.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork22-2022-01-01" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2021-07-09        | 2.4     | Malika     | Updated connection string  |
| 2021-05-19        | 2.3     | Lakshmi Holla |Updated question     |
| 2021-04-20        | 2.2     | Malika     | Added the libraries                |
| 2020-11-27        | 2.1     | Sannareddy Ramesh    | Modified data sets and added new problems |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
